In [1]:
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc, log_loss

In [2]:
with open('alexa_reviews_clean.pkl','rb') as read_file:
    df = pickle.load(read_file)

In [3]:
df.head()

,rating,date,variation,verified_reviews,feedback,review_length,new_reviews,sentiment
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1,13,love echo,positive
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1,9,love,positive
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1,195,sometimes play game answer question correctly ...,positive
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1,172,lot fun thing 4 yr old learn dinosaur control ...,positive
4,5,31-Jul-18,Charcoal Fabric,Music,1,5,music,positive


## Variations

In [4]:
df['variation'].value_counts()

Black  Dot                      516
Charcoal Fabric                 430
Configuration: Fire TV Stick    350
Black  Plus                     270
Black  Show                     265
Black                           261
Black  Spot                     241
White  Dot                      184
Heather Gray Fabric             157
White  Spot                     109
White                            91
Sandstone Fabric                 90
White  Show                      85
White  Plus                      78
Oak Finish                       14
Walnut Finish                     9
Name: variation, dtype: int64

In [5]:
df=df[df.variation!='Configuration: Fire TV Stick']
df['variation'].value_counts()

Black  Dot              516
Charcoal Fabric         430
Black  Plus             270
Black  Show             265
Black                   261
Black  Spot             241
White  Dot              184
Heather Gray Fabric     157
White  Spot             109
White                    91
Sandstone Fabric         90
White  Show              85
White  Plus              78
Oak Finish               14
Walnut Finish             9
Name: variation, dtype: int64

In [6]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 20)


/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
df[df['variation']=='White'].sample(10)

In [ ]:
# CHANGE VARIATION NAMES TO NAME OF ECHO MODELS

# ECHO 2nd Gen - charcoal fabric, heather gray fabric, 
# sandstone fabric, oak finish, walnut finish
df['variation']= df['variation']

# ECHO DOT - black dot, white dot, black, white

# ECHO SHOW - black show, white show

# ECHO PLUS - black plus, white plus

# ECHO SPOT - black spot, white spot


In [13]:
df['model']=np.where(df.variation.str.contains('Charcoal Fabric ') |
                     df.variation.str.contains('Heather Gray Fabric ') |
                     df.variation.str.contains('Sandstone Fabric ') |
                     df.variation.str.contains('Oak Finish ') |
                     df.variation.str.contains('Walnut Finish '),'echo',df['variation'])


In [ ]:
df['model']=np.where(df.variation.str.contains('Black  Dot') |
                    df.variation.str.contains('White  Dot') |
                    df.variation.str.contains('Black') |
                    df.variation.str.contains('White'), 'echo dot', df['model'])

In [ ]:
df['model']=np.where(df.variation.str.contains('Black  Show') |
                    df.variation.str.contains('White  Show'), 'echo show', df['model'])

In [ ]:
df['model']=np.where(df.variation.str.contains('Black  Plus') |
                    df.variation.str.contains('White  Plus'), 'echo plus', df['model'])


In [ ]:
df['model']=np.where(df.variation.str.contains('Black  Spot') |
                    df.variation.str.contains('White  Spot'), 'echo spot', df['model'])


In [8]:
df.sample(10)

,rating,date,variation,verified_reviews,feedback,review_length,new_reviews,sentiment,model
738,5,30-Jul-18,Charcoal Fabric,Tried to play certain broadway shows like Camelot and it gives ne the group Camelot.,1,84,try play certain broadway show like camelot give ne group camelot,positive,Charcoal Fabric
21,5,30-Jul-18,Charcoal Fabric,"We love Alexa! We use her to play music, play radio through iTunes, play podcasts through Anypod, and set reminders. We listen to our flash briefing of news and weather every morning. We rely on our custom lists. We like being able to voice control the volume. We're sure we'll continue to find new uses.Sometimes it's a bit frustrating when Alexa doesn't understand what we're saying.",1,385,love alexa use play music play radio itune play podcast anypod set reminder listen flash briefing news weather every morning rely custom list like able voice control volume sure well continue find new usessometime bit frustrating alexa do not understand say,positive,Charcoal Fabric
1963,5,25-Jul-18,Black Plus,No problems whatsoever makes life easy and sounds great.,1,56,problem whatsoever make life easy sound great,positive,echo plus
410,5,15-Jul-18,Black,I love it works very well,1,25,love work well,positive,Black
1760,4,30-Jul-18,White Plus,I did not know it needed to be plugged in to work. So far my experiences have been positive.,1,92,know need plugged work far experience positive,positive,echo plus
2724,5,30-Jul-18,Black Dot,"AMAZING. It has helped me become better organized, gives me traffic updates, weather updates, and helps me maintain a good, fast morning routine.",1,145,amazing help become well organized give traffic update weather update help maintain good fast morning routine,positive,Black Dot
1109,4,30-Jul-18,White Spot,Echo Spot works well except it does not work with the Ring doorbell as it states on the website. Since they were sold as a pair I thought that it would be app ready. Unfortunately that is not case. Perhaps Ring will update so that they can be integrated.,1,256,echo spot work well except work ring doorbell states website since sell pair thought would app ready unfortunately case perhaps re update integrate,positive,White Spot
2493,4,30-Jul-18,Black Dot,"I've only got to use it for a couple of weeks before having to go into the hospital, but had good time with it for the short time I did use it. It was fun trying to use Alexa",1,174,-PRON- have get use couple week go hospital good time short time use fun try use alexa,positive,Black Dot
2929,5,30-Jul-18,White Dot,"We really love our Echo Dot, we can get questions answered right away and play any music we can think of.",1,105,really love echo dot get question answer right away play music think,positive,White Dot
1513,5,30-Jul-18,Black Show,,1,1,,positive,Black Show


In [ ]:
df[df['variation']=='Black  Show']

# Corpus

In [ ]:
# CREATE DICTIONARY TO COUNT THE WORDS
count_dict_alex = {}

for doc in df['new_reviews']:
    for word in doc.split():
        if word in count_dict_alex.keys():
            count_dict_alex[word] +=1
        else:
            count_dict_alex[word] = 1
            
for key, value in sorted(count_dict_alex.items(), key=lambda item: item[1]):
    print("%s: %s" % (key, value))

In [ ]:
# REMOVE WORDS THAT OCCUR LESS THAN 10 TIMES
low_value = 10
bad_words = [key for key in count_dict_alex.keys() if count_dict_alex[key] < low_value]

In [ ]:
# CREATE A LIST OF LISTS - EACH DOCUMENT IS A STRING BROKEN INTO A LIST OF WORDS
corpus = [doc.split() for doc in df['new_reviews']]
clean_list = []
for document in corpus:
    clean_list.append([word for word in document if word not in bad_words])

In [ ]:
clean_list

In [ ]:
# USE CLEAN_LIST TO CREATE CORPUS
corpus=[]
for item in clean_list:
    item = ' '.join(item)
    corpus.append(item)
    
corpus

# LDA

In [ ]:
import gensim
from gensim import corpora
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
from nltk.corpus import stopwords

In [ ]:
clean_list[0][:5]

In [ ]:
# CREATE THE INPUTS OF LDA MODEL: DICTIONARY AND CORPUS
corpora_dict = corpora.Dictionary(clean_list)
corpus = [corpora_dict.doc2bow(line) for line in clean_list]

In [ ]:
# TRAIN THE LDA MODEL
lda_model = LdaModel(corpus=corpus,
                         id2word=corpora_dict,
                         random_state=100,
                         num_topics=3,
                         passes=5,
                         per_word_topics=True)

# See the topics
lda_model.print_topics(-1)

# Word2Vec

In [ ]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

In [ ]:
# Train Word2Vec model. Defaults result vector size = 100
model = Word2Vec(clean_list, min_count = 0, workers=cpu_count())

# Get the word vector for given word
model['sound']

model.most_similar('sound')

# Bigram

In [ ]:
dct = corpora.Dictionary(clean_list)
corpus = [dct.doc2bow(line) for line in clean_list]

# Build the bigram models
bigram = gensim.models.phrases.Phrases(clean_list, min_count=3, threshold=10)

# Construct bigram
print(bigram[clean_list[1]])

# Vader

In [ ]:
analyser = SentimentIntensityAnalyzer()

In [ ]:
def sentimentScore(sentences):
    analyzer = SentimentIntensityAnalyzer()
    results = []
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        print(str(vs))
        results.append(vs) 
    return results

In [ ]:
sentiment = sentimentScore(df['new_reviews'])


In [ ]:
sentiment_df = pd.DataFrame(sentiment)
sentiment_df.head()

In [ ]:
# align index to copy rating column for joining
df.index = sentiment_df.index
sentiment_df['rating'] = df['rating']
echo_vader = pd.concat([df, sentiment_df], axis=1)
echo_vader.head()

In [ ]:
#postive sentiment
color = ['#63ace5']
ax = echo_vader.groupby("variation").pos.mean().plot.bar(color = color, figsize = (9, 6))

plt.title('Positive Sentiment', fontsize = 20, weight='bold')

# plt.xlabel('Variation', fontsize = 16, weight='bold')
plt.xticks(rotation='90', fontsize=14, weight='bold')
ax.xaxis.label.set_visible(False)

plt.ylabel('Sentiment Rating', fontsize=16, weight='bold')
ax.set_ylim([0,0.5])
plt.yticks(fontsize=14)


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)


fig = plt.gcf()
plt.show()
plt.draw()

fig.savefig('postive_sentiment.jpg')
;

In [ ]:
#postive sentiment
color = ['#005b96']
ax = echo_vader.groupby("variation").neg.mean().plot.bar(color = color, figsize = (9, 6))

plt.title('Negative Sentiment', fontsize = 20, weight='bold')

# plt.xlabel('Variation', fontsize = 16, weight='bold')
plt.xticks(rotation='90', fontsize=14, weight='bold')
ax.xaxis.label.set_visible(False)

plt.ylabel('Sentiment Rating', fontsize=16, weight='bold')
ax.set_ylim([0,0.5])
plt.yticks(fontsize=14)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)


fig = plt.gcf()
plt.show()
plt.draw()

fig.savefig('negative_sentiment.jpg')
;

In [ ]:
group = df.groupby('rating').count()
group['date']

color = plt.cm.bone(np.linspace(0, 1, 6))
ax = group['date'].plot.bar(color='#7c86ac', figsize = (10, 6))

plt.title('Echo Ratings', fontsize = 20, weight='bold')
plt.xlabel('Ratings', fontsize = 16, weight='bold')
plt.ylabel('Count', fontsize=16, weight='bold')

plt.xticks(rotation='0', fontsize=14)
plt.yticks(fontsize=14)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

plt.show()

#change colour

# Sentiment Features

In [ ]:
neg_alexa = df[df['sentiment']=='negative']
pos_alexa = df[df['sentiment']=='positive']

## Negative

In [ ]:
from sklearn.feature_selection import chi2

tfidf_n = TfidfVectorizer(ngram_range=(2, 2))
X_tfidf_n = tfidf_n.fit_transform(neg_alexa['new_reviews'])
y_n = neg_alexa['rating']
chi2score_n = chi2(X_tfidf_n, y_n)[0]

In [ ]:
scores = list(zip(tfidf_n.get_feature_names(), chi2score_n))
chi2_n = sorted(scores, key=lambda x:x[1])
topchi2_n = list(zip(*chi2_n[-10:]))
x_n=range(len(topchi2_n[1]))

fig, ax = plt.subplots(figsize=(16,9))
ax.barh(x_n, topchi2_n[1], align='center', alpha=1, color='salmon')

plt.title('Alexa Negative Feedback', fontsize=24, weight='bold')

# x-axis
plt.xlabel("Feature Score", fontsize=22, weight='bold')
plt.xticks(fontsize=18)

#y-axis
labels = topchi2_n[0]
plt.yticks(x_n, labels, fontsize=18)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

fig = plt.gcf()
plt.show()
plt.draw()

fig.savefig('alexa_neg.jpg')
;

## Positive

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(2, 2))
X_tfidf = tfidf.fit_transform(pos_alexa['new_reviews'])
y = pos_alexa['rating']
chi2score = chi2(X_tfidf, y)[0]

In [ ]:
scores = list(zip(tfidf.get_feature_names(), chi2score))
chi2 = sorted(scores, key=lambda x:x[1])
topchi2 = list(zip(*chi2[-10:]))
x=range(len(topchi2[1]))

fig, ax = plt.subplots(figsize=(16,9))
ax.barh(x,topchi2[1], alpha=1, color='darkseagreen')

plt.title('Alexa Postive Feedback', fontsize=24, weight='bold')

# x-axis
plt.xlabel("Feature Score", fontsize=22, weight='bold')
plt.xticks(fontsize=18)

#y-axis
labels = topchi2[0]
plt.yticks(x, labels, fontsize=18)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

fig = plt.gcf()
plt.show()
plt.draw()

fig.savefig('alexa_positive.jpg')
;